In [1]:
# Lectura de archivos y Funciones recurrentes
import pandas as pd
import numpy as np

# Muestra todas las columnas del DataFrame
pd.set_option('display.max_columns', None)

# Función para convertir "M:SS.sss" a timedelta
def parse_mmss(x):
    if pd.isna(x):
        return pd.NaT
    parts = str(x).split(':')
    if len(parts) != 2:
        return pd.NaT
    mins, secs = parts
    try:
        total_secs = int(mins) * 60 + float(secs)
    except ValueError:
        return pd.NaT
    return pd.to_timedelta(total_secs, unit='s')

In [2]:
#Lectura y Carga de dataframes
circuits                = pd.read_csv('./00_data_raw/circuits.csv',                 index_col= 0)
constructor_results     = pd.read_csv('./00_data_raw/constructor_results.csv',      index_col= 0)
constructor_standings   = pd.read_csv('./00_data_raw/constructor_standings.csv',    index_col= 0)
constructors            = pd.read_csv('./00_data_raw/constructors.csv',             index_col= 0)
driver_standings        = pd.read_csv('./00_data_raw/driver_standings.csv',         index_col= 0)
drivers                 = pd.read_csv('./00_data_raw/drivers.csv',                  index_col= 0)
lap_times               = pd.read_csv('./00_data_raw/lap_times.csv',                index_col= 0)
pit_stops               = pd.read_csv('./00_data_raw/pit_stops.csv',                index_col= 0)
qualifying              = pd.read_csv('./00_data_raw/qualifying.csv',               index_col= 0)
races                   = pd.read_csv('./00_data_raw/races.csv',                    index_col= 0)
results                 = pd.read_csv('./00_data_raw/results.csv',                  index_col= 0)
sprint_results          = pd.read_csv('./00_data_raw/sprint_results.csv',           index_col= 0)
status                  = pd.read_csv('./00_data_raw/status.csv',                   index_col= 0)

In [3]:
# Columnas a eliminar por cada dataframe
drop_cols = {
    'circuits':              ['url'],
    'constructor_standings': ['positionText'],
    'constructors':          ['constructorRef', 'url'],
    'driver_standings':      ['positionText'],
    'drivers':               ['code', 'url'],
    'lap_times':             ['milliseconds'],
    'pit_stops':             ['time', 'milliseconds'],
    'races':                 ['fp1_date','fp1_time','fp2_date','fp2_time',
                              'fp3_date','fp3_time','time','quali_time','sprint_time','url'],
    'results':               ['positionText','time','milliseconds'],
    'sprint_results':        ['time','milliseconds']
}
for name, df in [
    ('circuits', circuits),
    ('constructor_results', constructor_results),
    ('constructor_standings', constructor_standings),
    ('constructors', constructors),
    ('driver_standings', driver_standings),
    ('drivers', drivers),
    ('lap_times', lap_times),
    ('pit_stops', pit_stops),
    ('qualifying', qualifying),
    ('races', races),
    ('results', results),
    ('sprint_results', sprint_results),
    ('status', status),
]:
    if name in drop_cols:
        df.drop(columns=drop_cols[name], inplace=True, errors='ignore')
    df.replace('\\N', np.nan, inplace=True)


In [4]:
# Formateo de columnas
# Texto
circuits['circuitRef']              = circuits['circuitRef'].str.capitalize()
constructors['nationality']         = constructors['nationality'].str.title().str.strip()
status['status']                    = status['status'].str.upper().str.strip()

# Fechas
races['date']                       = pd.to_datetime(races['date'], format='%Y-%m-%d',            errors='coerce').dt.strftime('%d-%m-%Y')
races['quali_date']                 = pd.to_datetime(races['quali_date'], format='%Y-%m-%d',      errors='coerce').dt.strftime('%d-%m-%Y')
races['sprint_date']                = pd.to_datetime(races['sprint_date'], format='%Y-%m-%d',     errors='coerce').dt.strftime('%d-%m-%Y')
drivers['dob']                      = pd.to_datetime(drivers['dob'], format='%Y-%m-%d',           errors='coerce').dt.strftime('%d-%m-%Y')

# Duraciones y tiempos
pit_stops.reset_index(inplace=True)
pit_stops['duration']               = pd.to_numeric(pit_stops['duration'],                  errors='coerce')
pit_stops['duration_s']             = pd.to_timedelta(pit_stops['duration'],                unit='s')

# Aplicamos la funcion parse_mmss a las columnas con tiempo tipo string
lap_times['lap_time']               = lap_times['time'].apply(parse_mmss)
qualifying['q1_time']               = qualifying['q1'].apply(parse_mmss)
qualifying['q2_time']               = qualifying['q2'].apply(parse_mmss)
qualifying['q3_time']               = qualifying['q3'].apply(parse_mmss)
results['fastestLapTime_td']        = results['fastestLapTime'].apply(parse_mmss)
sprint_results['fastestLapTime_td'] = sprint_results['fastestLapTime'].apply(parse_mmss)

In [5]:
# Creación de archivos CSV resultado del EDA
circuits.to_csv(                './01_data_EDA/circuits.csv',               index=False)
constructor_results.to_csv(     './01_data_EDA/constructor_results.csv',    index=False)
constructor_standings.to_csv(   './01_data_EDA/constructor_standings.csv',  index=False)
constructors.to_csv(            './01_data_EDA/constructors.csv',           index=False)
driver_standings.to_csv(        './01_data_EDA/driver_standings.csv',       index=False)
drivers.to_csv(                 './01_data_EDA/drivers.csv',                index=False)
lap_times.to_csv(               './01_data_EDA/lap_times.csv',              index=False)
pit_stops.to_csv(               './01_data_EDA/pit_stops.csv',              index=False)
qualifying.to_csv(              './01_data_EDA/qualifying.csv',             index=False)
races.to_csv(                   './01_data_EDA/races.csv',                  index=False)
results.to_csv(                 './01_data_EDA/results.csv',                index=False)
sprint_results.to_csv(          './01_data_EDA/sprint_results.csv',         index=False)
status.to_csv(                  './01_data_EDA/status.csv',                 index=False)

In [6]:
# Creación de tablas finales 
# Resultados de las carreras
gp_results = (
    results
    .merge(drivers[['forename','surname','nationality']],   left_on='driverId',         right_index=True)
    .merge(constructors[['name','nationality']],            left_on='constructorId',    right_index=True, suffixes=('_drv','_ctr'))
    .merge(races[['date','name','circuitId']],              left_on='raceId',           right_index=True, suffixes=('','_race'))
    .merge(circuits[['name','location','country']],         left_on='circuitId',        right_index=True, suffixes=('','_circuit'))
)

gp_results['date'] = pd.to_datetime(gp_results['date'],format='%d-%m-%Y',dayfirst=True,errors='coerce')
                            
# Puntos por piloto y temporada
driver_points = (
    gp_results
    .assign(year=gp_results['date'].dt.year)
    .groupby(['driverId','year'], as_index=False)['points']
    .sum()
)

# Puntos por escudería
constructor_points = (
    constructor_results
    .groupby('constructorId')['points']
    .sum()
    .reset_index()
)

# Paradas en boxes
pit_stops_clean = pit_stops[['raceId','driverId','stop','lap','duration_s']]

# Tiempos por vuelta
lap_times.reset_index(inplace=True)
lap_times_clean = lap_times[['raceId','driverId','lap','lap_time','position']]

# Resumen de la temporada
season_summary = (
    gp_results
    .assign(year=gp_results['date'].dt.year)
    .groupby(['year','driverId'])['points']
    .sum()
    .reset_index()
)

In [7]:
# Exportación a CSV de los nuevos archivos
gp_results.to_csv(          './02_data_clean/gp_results.csv',         index=False)
driver_points.to_csv(       './02_data_clean/driver_points.csv',          index=False)
constructor_points.to_csv(  './02_data_clean/constructor_points.csv',     index=False)
pit_stops_clean.to_csv(     './02_data_clean/pit_stops_clean.csv',        index=False)
lap_times_clean.to_csv(     './02_data_clean/lap_times_std.csv',          index=False)
season_summary.to_csv(      './02_data_clean/season_summary.csv',         index=False)